In [ ]:
import sys

sys.path.insert(0, "/home/leffff/PycharmProjects/LCT_Hack_Yakutiya_2023/venv/lib/python3.10/site-packages")

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig


MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."
# Ты Помогаешь жителям якутии и отвечаеншь на запросы жителей России о планах развития региона."


class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        do_sample=True,
        top_p=0.99,
        top_k=0,
        bos_token_id = 1,
        eos_token_id = 2,
        pad_token_id = 0,
        max_new_tokens=128, 
    )[0]
    print(data["input_ids"].device, model.device)
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

config = PeftConfig.from_pretrained(MODEL_NAME)
# config.top_k = 1

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
# generation_config.max_new_tokens = 512
# generation_config.top_k = 1
print(generation_config)

In [ ]:
%%time
inputs = [
    # "Планируется ли постройка много этажных гаражей, снос старых?", 
    # "Ответь на вопрос о развитии региона от лица мерии: За магазином Восток по Кирова ежегодно собирается большая лужа, стока воды нет, уходит под дома, боимся размытия сваи близ лежащих домов."
    # "Продолжится ли озеленение насыпей (искусственных гор) в близи города. Мне кажется, что это обязанность АЛРОСА?"
    "Почему так мало, а практически никак не озеленяют наш город, не озеленяют придомовые территории новых домов...Почему сейчас всё в приоритете стоянки для машин, а не площадки для детей, не безопасность жильцов и не зелёные насаждения. Знаю многие видя, что даже проявив свою инициативу, желая посадить самим около дома кусты или деревья…соседи выбегают и говорят, что против... И 2 вопрос. Вместо снесенных домов напротив магазина Темп, можно ли оставить все насаждения многолетние...и обыграть как маленький парк, но никак не автостоянки устраивать. Очень не хочется жить в каменных джунглях."
]

for inp in inputs:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print()
    print("==============================")
    print()
